# Collecting Weather Climate Data for period 2019

In this section, we collect the Weather Climate Data for the period of 2019. The data collected will act as part of the exogenous variables for the SARIMA forecast of the dengue cases for the 'unseen' 2019.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import re
import requests 
import pandas as pd
import pickle

In [2]:
driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')

/var/folders/t4/n89gfp4s10b4j476kq4xh2p00000gn/T/ipykernel_61508/3074833328.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('Users/wongh/Downloads/chromedriver_win32/chromedriver.exe')


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:
def get_html_table_as_df(table_xpath,df_to_save):
    '''This saves a html table as a df. 
    Rmb to create an empty df to save the data to before executing this function. 
    If the df to save the data to is not empty, this function appends cases'''
      
    # Use find_element_by_xpath method to locate an element
    table_html = driver.find_element(by=By.XPATH, value=table_xpath).get_attribute('outerHTML')
    
    data_table = pd.read_html(table_html)
    df = data_table[0]
    if df_to_save.empty:
        df_to_save = data_table[0]
    else: df_to_save = df_to_save.append(data_table[0])
    return df_to_save

In [ ]:
mss_url = 'http://www.weather.gov.sg/climate-historical-daily/'
driver.get(mss_url)
time.sleep(2)

In [ ]:
def button_dropdown_option_loop(option_xpath_id, number_of_options, display_button_xpath, 
                                table_xpath, df_to_save):
    '''This function selects the button and clicks on each option in order, starting from the top.
    Diplay button is then clicked to display the table below, and saves the table as a df'''
    for option in range(1,number_of_options+1):
        driver.find_element(by=By.XPATH, value=option_xpath_id).click()
        driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option)+']/a').click()
        driver.find_element(by=By.XPATH, value=display_button_xpath).click()
        time.sleep(2)
        df_to_save = get_html_table_as_df(table_xpath, df_to_save) 
        time.sleep(1)
    return df_to_save

In [ ]:
def button_dropdown_option(option_xpath_id, option_index, display_button_xpath):
    '''This function selects the button and clicks on a specific option.
    Index[1] is the first option from the top. 
    Diplay button is then clicked to display the table below'''
    driver.find_element(by=By.XPATH, value=option_xpath_id).click()
    driver.find_element(by=By.XPATH, value=option_xpath_id+'/ul/li['+str(option_index)+']/a').click()
    driver.find_element(by=By.XPATH, value=display_button_xpath).click()
    time.sleep(1)

In [ ]:
month_option_xpath_id = '//*[@id="monthDiv"]'
display_button_xpath = '//*[@id="display"]'
year_option_xpath_id = '//*[@id="yearDiv"]'

table_xpath = '//*[@id="temp"]/h4[2]/div/div/table'

### we are gonna do this one at a time because:
### a) this page loads very slowly
### b) we need to add a year column
### c) random errors seem to be happening (but seem to have disappeared when I increased the timer)
### d) I don't know how to create a loop that creates variabe names dynamically (e.)

In [ ]:
##2019 
df_temp_rain2019 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,2,display_button_xpath)
df_temp_rain2019 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2019)
df_temp_rain2019['Year'] = 2019

In [ ]:
df_temp_rain2019.head()


In [ ]:
# drop station as it is not relevant
df_temp_rain2019.isnull().sum()

In [ ]:
df_temp_rain2019.info()

In [ ]:
df_temp_rain2019 = df_temp_rain2019.replace('-', pd.np.nan)


In [ ]:
# Convert Mean Wind Speed to float
df_temp_rain2019['Mean Wind Speed (km/h)'] = df_temp_rain2019['Mean Wind Speed (km/h)'].astype(float)

In [ ]:
# impute median values due to values being right-skewed
df_temp_rain2019['Mean Wind Speed (km/h)'].fillna(df_temp_rain2019['Mean Wind Speed (km/h)'].median(), inplace=True)

In [ ]:
df_temp_rain2019['Date'].info()

In [ ]:
# Convert the date column into a datetime column
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['Date'] + ' ' + df_temp_rain2019['Year'].astype(str), format='%d %b %Y')

In [ ]:
df_temp_rain2019.head(10)

In [ ]:
# convert the date column to datetime
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['date'])

# sort the dataframe by date in descending order
df_temp_rain2019 = df_temp_rain2019.sort_values('date', ascending=True)

# reset the index to start at 0
df_temp_rain2019 = df_temp_rain2019.reset_index(drop=True)

In [ ]:
df_temp_rain2019.head()

In [ ]:
df_temp_rain2019.drop(columns=['Year','Date'], inplace=True)

In [ ]:
df_temp_rain2019.head()

In [ ]:
# Convert the 'date' column to a datetime format
df_temp_rain2019['date'] = pd.to_datetime(df_temp_rain2019['date'])

# Set the 'date' column as the index of the DataFrame
df_temp_rain2019 = df_temp_rain2019.set_index('date')

# Resample the DataFrame to a weekly frequency, calculating the mean of the seven daily readings for each week
weekly_df2019 = df_temp_rain2019.resample('W-MON', origin='start').mean().round(2)

# Reset the index of the resulting DataFrame to make the 'date' column a regular column again
weekly_df2019 = weekly_df2019.reset_index()


In [ ]:
weekly_df2019.head()

## Exporting the Weather Climate Data 2019

In [ ]:
# Creating temp_rain.csv
weekly_df2019.to_csv('../../assets/weekly_rain2019.csv')